In [ ]:
# default_exp core

In [ ]:
#hide
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#hide
!pip install fastcore
from fastcore.all import *
%load_ext autoreload 
%autoreload 2
%cd "/content/drive/MyDrive/Coding/ModelAssistedLabel"

     |████████████████████████████████| 61kB 3.5MB/s 
/content/drive/MyDrive/Coding/ModelAssistedLabel


# AutoWeights
> Prepare files in repository to run the `train.py` file.

Assuming use of the [YOLOv5 format](https://github.com/AlexeyAB/Yolo_mark/issues/60).

In [ ]:
# export
from ModelAssistedLabel.core import Trainer, Generation, Defaults
from datetime import datetime

class AutoWeights():
  """
  Given a bag of images (.jpg) and labels (.txt) in YOLOv5 format in a repository,
  initialize the ROOT directory with a train-valid-test split and a file needed 
  by the Ultralytics repository. Pairs are identified if `image[:-4] == label[:-4]`

  Then call `generate_weights` to run `train.py`. The resultant file will try to 
  be moved to the `out_dir` and if a conflict exists, a new name will be made.
  """
  def __init__(self, resource_dir, name="DEFAULT", out_dir=".", MAX_SIZE=5, custom_data_yaml=None, verbose=True, train_path = "yolov5/runs/train"):
    """
    Args:
      resource_dir: location of the bag of images/labels
      out_dir: where the results of train.py are moved
      MAX_SIZE: parameter for `Generation`
      custom_data_yaml: see `Defaults`'s `data_yaml` for the default value
      verbose: Print summary information
      train_path: path to Ultralytic's default output folder
    """
    self.resource_dir = resource_dir
    self.name = name
    self.resource_paths = ["test/", "train/", "valid/", "data.yaml"]
    for r in self.resource_paths: #make sure none of these paths already exist
      assert os.path.exists(r) is False, f"{r} exists... You may be in the middle of an active project"
    self.out_dir = out_dir
    self.train_path = train_path
    #automatically build the resource paths and prepare for traniing
    self.__prepare_split__(MAX_SIZE=MAX_SIZE, data_yaml=custom_data_yaml, verbose=verbose)
    assert self.g is not None

  def generate_weights(self, epochs, tidy_weights=True):
    """
    Creates a `Trainer` object and trains for a given amount of time.

    Args:
      epochs: number of iterations (according to docs, over 3000 is not uncommon)
      tidy_weights: if True, remove all of the resources in `self.resources`
    
    Returns:
      path to the output folder of train.py
    """
    t = Trainer(self.name)
    ldir = lambda path: set(os.listdir(path))

    before = ldir(self.train_path)
    t.train(epochs)
    after = ldir(self.train_path)
    
    test_eq(len(after), len(before)+1) #only should have made one new file
    diff = list(after - before)[0]

    results_path = os.path.join(self.train_path, diff)

    if tidy_weights:
      results_path = self.__tidy_weights__(results_path = results_path)

    self.__cleanup__()
    self.last_results_path = results_path
    return results_path

  def __prepare_split__(self, MAX_SIZE, data_yaml, verbose):
    """
    Gets the local filesystem ready to run the wrapper for "train.py".

    Args:
      MAX_SIZE:
      data_yaml:
      verbose: print summary information for the split
    """  
    if data_yaml is None:
      data_yaml = Defaults().data_yaml
    self.g = Generation(repo=self.resource_dir, out_dir=self.out_dir, data_yaml=data_yaml)
    self.g.set_split(MAX_SIZE=MAX_SIZE)
    self.__split_and_organize_folders__(verbose=verbose)
  
  def __split_and_organize_folders__(self, verbose):
    """
    Assume zip file contains 4 items:
      * data.yaml
      * train/
      * valid/
      * test/

    Extract these 4 resources to the ROOT directory and remove the original 
    part of the file.

    Args:
      verbose: print summary information about the split
    """
    assert self.g is not None

    def tostr(split):
      return [{k: len(v)} for k,v in split.items()]
    def sumg(split):
      return sum([list(x.values())[0] for x in tostr(split)])
    if verbose:
      print("summary: ", tostr(self.g.split))
      print("checksum:", sumg(self.g.split))

    zipped = self.g.process_split(self.name) #create a zip file in the ROOT directory
    local = os.path.basename(zipped)
    os.system(f'unzip "{local}"') #grab data
    #move the contents of the zip file into postion within the ROOT directory
    for content in self.resource_paths:
      os.system(f"mv '{local[:-4]}/{content}' .")
    #remove zip file
    os.system(f"rm -f -r '{local}'")
    #removed the folder that was taken out of the zip
    os.system(f"rm -f -r '{local[:-4]}'")

  def __cleanup__(self):
    """
    Removes all resources in `self.resource_paths` from the filesystem.
    """
    for r in self.resource_paths:
      os.system(f"rm -f -r {r}")

  def __tidy_weights__(self, results_path):
    """
    Moves the results to a desired directly while ensuring that no data is overwritten

    Args:
      results_path: path to the folder that has desired information
    
    Returns:
      Path to the newly-moved results
    """      
    while True:
      now = datetime.now()
      out = f"{os.path.basename(results_path)}-{now.strftime('%f')}" #basename + "random" string
      outfolder = os.path.join(self.out_dir, out)
      if not os.path.exists(outfolder): #only stops if a unique name has been found
        break
    os.system(f"mv '{results_path}' '{outfolder}'")
    return outfolder

In [ ]:
repo = "/content/drive/MyDrive/Coding/Roboflow Export (841)"
name = "nospaces"
wm = AutoWeights(repo, name)

summary:  [{'train': 4}, {'valid': 1}, {'test': 0}]
checksum: 5
target/dest /content/drive/MyDrive/Coding/Roboflow Export (841)/images/digittake-66-jpg_jpg.rf.95253009459f7f68fc3d6c70ee0e598f.jpg | ./train/images/digittake-66-jpg_jpg.rf.95253009459f7f68fc3d6c70ee0e598f.jpg
target/dest /content/drive/MyDrive/Coding/Roboflow Export (841)/labels/digittake-66-jpg_jpg.rf.95253009459f7f68fc3d6c70ee0e598f.txt | ./train/labels/digittake-66-jpg_jpg.rf.95253009459f7f68fc3d6c70ee0e598f.txt
target/dest /content/drive/MyDrive/Coding/Roboflow Export (841)/images/save_dirrsave_dirr0f7d9a795212d9e140181ec73a32a493af22a100-jpg-jpg_jpg.rf.d5c3022148a7eadc95b5f1adf84948f6.jpg | ./train/images/save_dirrsave_dirr0f7d9a795212d9e140181ec73a32a493af22a100-jpg-jpg_jpg.rf.d5c3022148a7eadc95b5f1adf84948f6.jpg
target/dest /content/drive/MyDrive/Coding/Roboflow Export (841)/labels/save_dirrsave_dirr0f7d9a795212d9e140181ec73a32a493af22a100-jpg-jpg_jpg.rf.d5c3022148a7eadc95b5f1adf84948f6.txt | ./train/labels/save_di

In [ ]:
%%time 
wm.generate_weights(10)

CPU times: user 3.51 ms, sys: 12.9 ms, total: 16.4 ms
Wall time: 1min 13s


'./nospaces7-896351'

In [ ]:
%run "_Synch.ipynb"

/content/drive/MyDrive/Coding/ModelAssistedLabel
install nbdev: yes
build resources: yes
Converted 00_ultralytics.ipynb.
Converted 01_split.ipynb.
Converted 02_train.ipynb.
Converted 03_make_weights.ipynb.
Converted index.ipynb.
converting: /content/drive/My Drive/Coding/ModelAssistedLabel/03_make_weights.ipynb
converting /content/drive/My Drive/Coding/ModelAssistedLabel/index.ipynb to README.md
git commit -m: AutoWeights up and running!

Executing: git config --local include.path ../.gitconfig
Success: hooks are installed and repo's .gitconfig is now trusted
